In [8]:
import os
import pandas as pd

In [9]:
from collections import defaultdict as dd

rec_dd = lambda: dd(rec_dd)

def dd_to_dict(d):
    if isinstance(d, dd):
        d = {k: dd_to_dict(v) for k, v in d.items()}
    return d

In [10]:
tr_dir = "trfiles"

tr_names = [
    "crossbar_64_bt.W.64_trace_1.00e09_4096_620085_62590300.tr",
    "crossbar_64_cg.W.64_trace_1.00e09_4096_267004_41961500.tr",
    "crossbar_64_ep.W.64_trace_1.00e09_4096_2655_9327740.tr",
    "crossbar_64_ft.W.64_trace_1.00e09_4096_18223_2379540.tr",
    "crossbar_64_is.W.64_trace_1.00e09_4096_53946_4909650.tr",
    "crossbar_64_lu.W.64_trace_1.00e09_4096_2163744_243721000.tr",
    "crossbar_64_mg.W.64_trace_1.00e09_4096_72726_5670700.tr",
    "crossbar_64_sp.W.64_trace_1.00e09_4096_1234359_134421000.tr"
]

apps = ["BT", "CG", "EP", "FT", "IS", "LU", "MG", "SP"]

tr_app = dict(zip(tr_names, apps))

sample_period = 100000
cycle_margin = 10000

summary = rec_dd()

for tr_name in tr_names:
    tmp_app = tr_app[tr_name]
    summary[tmp_app]["app"] = tmp_app
    summary[tmp_app]["num_packets"] = 0
    summary[tmp_app]["num_flits"] = 0

    tr_path = os.path.join(tr_dir, tr_name)
    with open(tr_path, mode="r") as f:
        is_head = True
        for line in f:
            row = list(map(int, line.split()))
            if is_head:
                print(row)
                summary[tmp_app]["num_packets"] = row[0]
                is_head = False
            else:
                cycle, src, dst, flits = row
                summary[tmp_app]["num_flits"] += flits

    trace_cycles = cycle
    max_samples = (trace_cycles + cycle_margin) // sample_period + 1
    summary[tmp_app]["sim_cycles"] = sample_period * max_samples
                # exit(1)

summary = dd_to_dict(summary)
# max_samples = (trace_cycles + cycle_margin) // sample_period + 1


[620085]
[267004]
[2655]
[18223]
[53946]
[2163744]
[72726]
[1234359]


In [11]:

df = pd.DataFrame(summary)
df = df.transpose()
print(df)

df2 = df.drop("app", axis=1)

s = df2.style
s.format('{:,}'.format)
print(s.to_latex())
# num_nodes = 64

# df["injection_rate"] = df.apply(lambda x: x["num_flits"] / num_nodes / x["sim_cycles"], axis=1)

# print(df)



   app num_packets num_flits sim_cycles
BT  BT      620085    697653   62700000
CG  CG      267004    370172   42000000
EP  EP        2655      2655    9400000
FT  FT       18223     25903    2400000
IS  IS       53946     58874    5000000
LU  LU     2163744   2308448  243800000
MG  MG       72726     84886    5700000
SP  SP     1234359   1618884  134500000
\begin{tabular}{llll}
 & num_packets & num_flits & sim_cycles \\
BT & 620,085 & 697,653 & 62,700,000 \\
CG & 267,004 & 370,172 & 42,000,000 \\
EP & 2,655 & 2,655 & 9,400,000 \\
FT & 18,223 & 25,903 & 2,400,000 \\
IS & 53,946 & 58,874 & 5,000,000 \\
LU & 2,163,744 & 2,308,448 & 243,800,000 \\
MG & 72,726 & 84,886 & 5,700,000 \\
SP & 1,234,359 & 1,618,884 & 134,500,000 \\
\end{tabular}

